In [ ]:
!python -m pip install git+https://github.com/huggingface/optimum.git#egg=optimum[onnxruntime]

In [ ]:
!pip install evaluate[evaluator] mkl-include mkl

In [ ]:
from optimum.onnxruntime import ORTModelForSeq2SeqLM
from transformers import BartTokenizerFast
from pathlib import Path

checkpoint = "facebook/bart-large-cnn"
onnx_path = Path("onnx")

# Load vanilla transformers and convert to onnx
model = ORTModelForSeq2SeqLM.from_pretrained(checkpoint, from_transformers=True)
tokenizer = BartTokenizerFast.from_pretrained(checkpoint)

# save onnx checkpoint and tokenizer
model.save_pretrained(onnx_path)
tokenizer.save_pretrained(onnx_path)

Optimum allows you yo run ONNX models with the pipeline() function from 🤗 Transformers. 

In [ ]:
from transformers import pipeline


text = """
Solar physics is the branch of astrophysics that specializes in the study of the Sun.
It deals with detailed measurements that are possible only for our closest star.It intersects with many disciplines of pure physics, astrophysics, and computer science, 
including fluid dynamics, plasma physics including magnetohydrodynamics, seismology, particle physics, atomic physics, nuclear physics, 
stellar evolution, space physics, spectroscopy, radiative transfer, applied optics, signal processing, computer vision, computational physics, stellar physics and solar astronomy.
Because the Sun is uniquely situated for close-range observing (other stars cannot be resolved with anything like the spatial or temporal resolution that the Sun can), 
there is a split between the related discipline of observational astrophysics (of distant stars) and observational solar physics. The study of solar physics is also important 
as it provides a "physical laboratory" for the study of plasma physics.Babylonians were keeping a record of solar eclipses, 
with the oldest record originating from the ancient city of Ugarit, in modern-day Syria.This record dates to about 1300 BC.
Ancient Chinese astronomers were also observing solar phenomena \n(such as solar eclipses and visible sunspots) with the purpose of keeping track of calendars, 
which were based on lunar and solar cycles.\nUnfortunately, records kept before 720 BC are very vague and offer no useful information.
However, after 720 BC, 37 solar eclipses were noted over the course of 240 years.Astronomical knowledge flourished in the Islamic world during medieval times.
Many observatories were built in cities from Damascus to Baghdad, where detailed astronomical observations were taken.Particularly, a few solar parameters were measured 
and detailed observations of the Sun were taken.
"""
vanilla_clf = pipeline("summarization", model=model, tokenizer=tokenizer)
vanilla_clf(text)

In [ ]:
from optimum.onnxruntime import ORTOptimizer
from optimum.onnxruntime.configuration import OptimizationConfig

# create ORTOptimizer and define optimization configuration
optimizer = ORTOptimizer.from_pretrained(checkpoint, feature=model.pipeline_task)
optimization_config = OptimizationConfig(optimization_level=99) # enable all optimizations

# apply the optimization configuration to the model
optimizer.export(
    onnx_model_path = onnx_path / "model.onnx",
    onnx_optimized_model_output_path = onnx_path / "model-optimized.onnx",
    optimization_config=optimization_config,
)

In [ ]:
del model, optimizer

In [ ]:
from optimum.onnxruntime import ORTModelForSeq2SeqLM
from transformers import pipeline

# Load optimized model
model = ORTModelForSeq2SeqLM.from_pretrained(onnx_path, file_name="model-optimized.onnx")

# create optimized pipeline
optimized_clf = pipeline("summarization", model=model, tokenizer=tokenizer)
optimized_clf(text)